**User Input and Image Generation**

Steps with Gradio:
1. Ask User for Image Concepts: Use Gradio to create a text input interface where users can enter their image concepts.
2. Clean User Input: Ensure the prompt is clean of unnecessary spaces or characters.
3. Prompt the Image Generation Agent: Call APIs to generate images using Stable Diffusion XL and DALL-E based on user input.

In [1]:
# Import Dependencies
import os
from dotenv import load_dotenv

import requests
import openai
import base64
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

import gradio as gr


In [2]:
# Load environment variables.
load_dotenv()

# Store the API key in a variable.
openai.api_key = os.getenv("OPENAI_API_KEY")
STABILITY_AI_API_KEY = os.getenv("STABILITY_AI_API_KEY")

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"

In [3]:
# Utility to view images for debugging
def view_base64_image(base64_string):
    """Decodes a base64 encoded image and displays it using matplotlib."""

    # Decode the base64 string
    image_data = base64.b64decode(base64_string)

    # Open the image using PIL
    image = Image.open(io.BytesIO(image_data))

    # Display the image
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [4]:
# Utility to clean user input by removing extra spaces
def clean_user_input(user_query):
    cleaned_query = ' '.join(user_query.split())
    return cleaned_query

In [5]:
# Call Stability AI API and generate an image
def generate_stability_ai_image(prompt):
    host = 'https://api.stability.ai/v2beta/stable-image/generate/sd3'
    params = {
        "prompt" : prompt,
        "negative_prompt" : '',
        "aspect_ratio" : '1:1',
        "seed" : 0,
        "output_format" : 'jpeg',
        "model" : "sd3",
        "mode" : "text-to-image"
    }
    headers = {
        "Accept": "image/*",
        "Authorization": f"Bearer {STABILITY_AI_API_KEY}"
    }

    # Encode parameters
    files = {}
    image = params.pop("image", None)
    mask = params.pop("mask", None)
    if image is not None and image != '':
        files["image"] = open(image, 'rb')
    if mask is not None and mask != '':
        files["mask"] = open(mask, 'rb')
    if len(files)==0:
        files["none"] = ''

    # Send request
    print(f"Sending REST request to {host}...")
    response = requests.post(
        host,
        headers=headers,
        files=files,
        data=params
    )
    if not response.ok:
        raise Exception(f"HTTP {response.status_code}: {response.text}")

    return base64.b64encode(response.content)
    # To test the function: response = generate_stability_ai_image("cute shiba inu")


In [6]:
# Call Dall-E Open AI API and generate an image
def call_dalle_api(prompt):
    client = openai.OpenAI()
    response = client.images.generate(
    model="dall-e-2",
    prompt=prompt,
    size="512x512",
    quality="standard",
    n=1,
    response_format="b64_json"
    )

    return response.data[0].b64_json
    # To test the function: response = call_dalle_api("A realistic image of a shiba inu with a birthday hat on the street")

In [7]:
# Convert base64 string to a PIL Image object
def base64_to_pil_image(base64_string):

    # Decode the base64 string
    image_data = base64.b64decode(base64_string)

    # Create a BytesIO object from the decoded data
    image_bytes = BytesIO(image_data)

    # Open the image using PIL
    image = Image.open(image_bytes)

    return image

In [40]:
import time

# Generate images and assess their quality
def generate_images_with_quality(prompt):
    cleaned_prompt = clean_user_input(prompt)
    stability_ai_image = base64_to_pil_image(generate_stability_ai_image(cleaned_prompt))
    dalle_image = base64_to_pil_image(call_dalle_api(cleaned_prompt))

    timestamp = time.time()
    
    with open(f"generated_images/{timestamp}_prompt.txt", "w") as f:
        # Write text to the file
        f.write(f'{prompt}, {cleaned_prompt}')
        
    stability_ai_image.save(f'generated_images/{timestamp}_stability_ai.jpg')
    dalle_image.save(f'generated_images/{timestamp}_dalle.jpg')

    #sd_quality = assess_image_quality(sd_image)
    #dalle_quality = assess_image_quality(dalle_image)
    return stability_ai_image, dalle_image #, f"Quality: {sd_quality}", dalle_image, f"Quality: {dalle_quality}"

In [41]:
# Gradio interface setup for user interaction
iface = gr.Interface(fn=generate_images_with_quality,
                     inputs="text",
                     outputs=[gr.Image(type="pil", label="Stability AI Image"), gr.Image(type="pil", label="Dall-E Image")],
                     title="Text-to-Image Generation",
                     description="Input a concept to generate images using Stability AI and Dall-E.")

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7866
Running on public URL: https://9ec2ee4c85ff9c85cc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Sending REST request to https://api.stability.ai/v2beta/stable-image/generate/sd3...


**Image Preprocessing and Quality Assessment**

Steps:
1. Measure Image Quality: Preprocess images to evaluate their quality using a classifier.

In [10]:
# Import Dependencies
import numpy as np
import cv2
from sklearn.ensemble import RandomForestClassifier

In [11]:
# Preprocess the image for quality assessment
def preprocess_image(image):
    gray_image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    resized_image = cv2.resize(gray_image, (256, 256))  # Resize image
    normalized_image = resized_image / 255.0  # Normalize pixel values
    return normalized_image.flatten()

In [12]:
# Replace with actual trained model for real use case
image_quality_classifier = RandomForestClassifier()

In [13]:
# Assess the quality of an image using a classifier
def assess_image_quality(image):
    feature_vector = preprocess_image(image)
    quality_score = image_quality_classifier.predict([feature_vector])[0]
    return quality_score

**Image Evaluation, Relevance Check, and Description Generation**

Steps:
1. Evaluate Image Relevance: Use an LLM to assess whether the generated image is relevant to the prompt.
2. Generate Image Descriptions: Use a classifier to create descriptive summaries of the images.
3. Compare Models: Analyze how each model's output aligns with the user query.

In [14]:
# Import dependencies
import openai

# Imports for loading environment variables.
from dotenv import load_dotenv
import os

In [15]:
# Evaluate the relevance of an image using LLM
def evaluate_image_relevance(prompt, image):
    evaluation_result = llm_evaluate(prompt, image)
    relevance_score = compare_with_prompt(evaluation_result, prompt)
    return relevance_score

In [16]:
# Use LLM for evaluating relevance based on prompt and image data
def llm_evaluate(prompt, image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    response = openai.Completion.create(
      engine="text-davinci-003",
      prompt=f"Evaluate the relevance of this image based on the prompt: {prompt}. Image data: {img_str}",
      max_tokens=50
    )
    
    return response.choices[0].text.strip()

In [17]:
# Compare LLM evaluation result with original prompt
def compare_with_prompt(evaluation_result, prompt):
    return "Relevant" if "relevant" in evaluation_result.lower() else "Not Relevant"

In [18]:
# Generate description
def generate_description(image):
    # Placeholder for a model that generates descriptions from images
    description_model_output = "A detailed description of the image."
    return description_model_output

**Data Handling with Pandas**

Steps:
1. Store Results: Use Pandas to handle data and store results in a structured format for analysis.

In [19]:
# Import dependencies
import pandas as pd

In [20]:
# Create a DataFrame to store results of generation and evaluation
results_df = pd.DataFrame(columns=['Prompt', 'SD_Image_Quality', 'DALL_E_Image_Quality', 'SD_Relevance', 'DALL_E_Relevance'])

In [21]:
# Simulated values for debugging
prompt = "A futuristic cityscape"
sd_quality = 8
dalle_quality = 7
sd_relevance = "Relevant"
dalle_relevance = "Not Relevant"

# Adding data using pd.concat instead of append
new_row = pd.DataFrame([{ 
    'Prompt': prompt, 
    'SD_Image_Quality': sd_quality, 
    'DALL_E_Image_Quality': dalle_quality, 
    'SD_Relevance': sd_relevance,
    'DALL_E_Relevance': dalle_relevance
}])

results_df = pd.concat([results_df, new_row], ignore_index=True)

results_df

,Prompt,SD_Image_Quality,DALL_E_Image_Quality,SD_Relevance,DALL_E_Relevance
0,A futuristic cityscape,8,7,Relevant,Not Relevant


In [22]:
# Save results into CSV file for further analysis or reporting
results_df.to_csv('image_generation_results.csv', index=False)

Sending REST request to https://api.stability.ai/v2beta/stable-image/generate/sd3...
